# Recomendation system 

### Step 1: create test dataset 

features:
 - id: int 
 - name: string 
 - gender: categorcial -> ['M','F']
 - age: int ->     1<=age<=80
 - consumption_status: ordinal -> ['1','2','3']
 - sleeping_status: ordinal -> ['1','2','3','4','5']
 - height_cm: float    
 - weight_kg: float   
 - prefer_food: [list]

In [2]:
import numpy as np 
import pandas as pd 
import random as rd 
import string

In [8]:
_id = np.linspace(1,1000,1000,dtype = np.int16)

name = [] 
gender = [] 
age = [] 
consumption_status = [] 
sleeping_status = [] 
h_cm = []
w_kg = []

def get_random_name(length):
    letters = string.ascii_lowercase 
    result_str = ''.join(rd.choice(letters) for i in range(length))
    return result_str

for _ in range(1000): 
    name.append(get_random_name(8))
    gender.append(rd.choice(['M','F'])) 
    age.append(rd.randint(1,80))
    consumption_status.append(rd.choice(['1','2','3']))
    sleeping_status.append(rd.choice(['1','2','3','4','5']))
    
for i in range(1000):
    if age[i] >= 1 and age[i] <= 5: 
        h_cm.append(round(rd.uniform(76.2,101.6),2))
        w_kg.append(round(rd.uniform(12.25,19.05),2))
    elif age[i] >5 and age[i] <= 10:
        h_cm.append(round(rd.uniform(109.22,138.43),2))
        w_kg.append(round(rd.uniform(14.3,35.6),2))
    elif age[i] >10 and age[i] <= 13:
        h_cm.append(round(rd.uniform(149.10,156.21),2))
        w_kg.append(round(rd.uniform(25.2,49.9),2))
    elif age[i] >13 and age[i] <= 18:
        h_cm.append(round(rd.uniform(163.83,175.77),2))
        w_kg.append(round(rd.uniform(42.6,70.5),2))
    else: 
        if gender[i] == 'M':
            h_cm.append(round(rd.uniform(165,190),2))
            w_kg.append(round(rd.uniform(65,100),2))
        if gender[i] == 'F':
            h_cm.append(round(rd.uniform(155,180),2))
            w_kg.append(round(rd.uniform(50,80),2))   

In [9]:
# foods 
import requests 
url = "https://eatingatoz.com/food-list/"
response = requests.get(url)
response

<Response [200]>

In [10]:
from bs4 import BeautifulSoup 
soup = BeautifulSoup(response.content)
soup1 = soup.find('div',class_='entry')
food_pool = [] 
for j in soup1.find_all('ul'):
    for i in j.find_all('li'):
        food_pool.append(i.text)
food_pool.remove('')
food_pool.remove('? (any ideas)')
food_pool.remove('acorn squash \xa0(NOPE: this is actually an ‘S’)')
food_pool.remove('asian noodles (NOPE, an ‘N’)')
food_pool.remove('ahi tuna (NOPE: this is actually a ‘T’)')
food_pool.remove('fish \xa0(might be too vague)')
food_pool = np.array(food_pool)

In [11]:
prefer_food = {} 
for i in range(1,1001): 
    prefer_food[i] = list(food_pool[rd.sample(range(0,125),30)])

Form DataFrame as datasets 
 - two datasets: personal_information, prefered_food 
 - connect the two datasets by merge on user_id

In [14]:
"""
name = [] 
gender = [] 
age = [] 
consumption_status = [] 
sleeping_status = [] 
h_cm = []
w_kg = []
"""

personal_information = pd.DataFrame({'user_id' : list(_id),
                                   'name' : name,
                                   'gender' : gender,
                                   'age' : age,
                                   'consumption_status' : consumption_status,
                                   'sleeping_status' : sleeping_status,
                                   'height_cm' : h_cm,
                                   'weight_kg': w_kg})
pred_food = pd.DataFrame(prefer_food).transpose()
pred_food['user_id'] = _id

In [15]:
personal_information.head()

,user_id,name,gender,age,consumption_status,sleeping_status,height_cm,weight_kg
0,1,fffbnltz,M,79,1,4,174.03,83.01
1,2,ogmoxhpw,F,74,2,4,168.05,67.93
2,3,lvudatkj,F,39,1,4,164.47,59.86
3,4,hiatrrkg,F,40,1,3,177.81,55.27
4,5,kzypbgjc,F,15,1,5,169.12,66.82


In [16]:
pred_food.head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,user_id
1,black beans,kale,Quesadilla,arugala,catfish,Reuben,avacado,hot dogs,carrots,enchilada,...,curry,granola,kabobs,duck,Pizza,celery,Tater tots,honey,ginger,1
2,dips,bluefish,Irish stew,franks,Linguine,beer,applesauce,green beans,jelly / jam,barley,...,apples,Moose,dumplings,kiwi,fajita,bison,hot dogs,dates,Milk,2
3,Pepperoni,Guancamole,halibut,carrots,dips,cookies,Wine,cupcakes,ketchup,Moose,...,ham,Milkshake,fajita,Spaghetti,cake,Walnuts,avacado,babaganoosh,cereal,3
4,eggrolls,hot dogs,Irish stew,Spaghetti,Bruscetta,avacado,bisque,Meatballs,gumbo,crab,...,Ostrich,Ziti,franks,Milkshake,cereal,English muffins,Pepperoni,Graham crackers,black beans,4
5,black beans,enchilada,Quiche,Pepperoni,barley,catfish,BBQ,Noodles,Moose,clams,...,cupcakes,edimame,franks,Yogurt,Waffles,coffee,cheese,Reuben,babaganoosh,5


In [17]:
personal_information.merge(pred_food)

,user_id,name,gender,age,consumption_status,sleeping_status,height_cm,weight_kg,0,1,...,20,21,22,23,24,25,26,27,28,29
0,1,fffbnltz,M,79,1,4,174.03,83.01,black beans,kale,...,alfalfa,curry,granola,kabobs,duck,Pizza,celery,Tater tots,honey,ginger
1,2,ogmoxhpw,F,74,2,4,168.05,67.93,dips,bluefish,...,Indian food,apples,Moose,dumplings,kiwi,fajita,bison,hot dogs,dates,Milk
2,3,lvudatkj,F,39,1,4,164.47,59.86,Pepperoni,Guancamole,...,English muffins,ham,Milkshake,fajita,Spaghetti,cake,Walnuts,avacado,babaganoosh,cereal
3,4,hiatrrkg,F,40,1,3,177.81,55.27,eggrolls,hot dogs,...,gnocchi,Ostrich,Ziti,franks,Milkshake,cereal,English muffins,Pepperoni,Graham crackers,black beans
4,5,kzypbgjc,F,15,1,5,169.12,66.82,black beans,enchilada,...,chowder,cupcakes,edimame,franks,Yogurt,Waffles,coffee,cheese,Reuben,babaganoosh
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,nzhfvobt,F,64,1,2,163.93,52.41,halibut,fajita,...,French dip,green beans,arugala,ice cream,kidney beans,Indian food,Ziti,fondu,chimichanga,jambalaya
996,997,nofhzslm,M,33,2,5,185.64,79.36,barley,Indian food,...,jambalaya,Pizza,jelly / jam,English muffins,kale,bagels,huenos rancheros,Milk,bacon,falafel
997,998,qnycpklc,F,16,2,3,172.40,57.85,curry,bagels,...,Venison,Milk,Reuben,Indian food,ham,enchilada,applesauce,Lamb,hash browns,Italian bread
998,999,ulmfwckq,F,61,2,3,157.77,69.57,Pizza,Milk,...,Apple juice,beer,antelope,Wine,chocolate,Graham crackers,Irish stew,Quesadilla,kabobs,cupcakes


In [18]:
personal_information.to_csv('personal_information.csv', index=False)
pred_food.to_csv('pred_food.csv',index=False)